In [68]:
import pandas as pd
import requests
import random
import json
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
from sqlalchemy import create_engine

In [3]:
db_connection_str = 'sqlite:///./raw_data_project_m1.db'
db_connection = create_engine(db_connection_str)

In [4]:
db_connection

Engine(sqlite:///./raw_data_project_m1.db)

In [5]:
db_connection.table_names()

['career_info', 'country_info', 'personal_info', 'poll_info']

In [6]:
pd.read_sql_query('select distinct(rural),count(rural) from country_info group by rural', db_connection)

,rural,count(rural)
0,Country,772
1,Non-Rural,1372
2,city,1664
3,countryside,482
4,rural,1517
5,urban,3842


In [7]:
career = pd.read_sql_table(table_name='career_info', con=db_connection)

In [8]:
#career.groupby('normalized_job_code').count()

In [9]:
#career['col_clone']=career['normalized_job_code']

In [10]:
response = requests.get('http://api.dataatwork.org/v1/jobs')

json_data = response.json()

In [11]:
ex=[]
for json_item in json_data:
    try:
        ex.append(json_item['title'])
    except:
        print('Mis cojones')
    

Mis cojones


In [12]:
df=pd.read_sql_query('select uuid, normalized_job_code from career_info', db_connection)

In [13]:

jobs_id_list =[]
codelist_df = df['normalized_job_code'].unique().tolist()
    
for job_code in codelist_df:
    req= requests.get(f'http://api.dataatwork.org/v1/jobs/{job_code}')
    jobs_id_list.append(req.json())
    
jobs_id_list   

[{'error': {'code': 404, 'message': 'Cannot find job with id None'}},
 {'uuid': '861a9b9151e11362eb3c77ca914172d0',
  'title': 'Automatic Data Processing Planner',
  'normalized_job_title': 'automatic data processing planner',
  'parent_uuid': '0148f61d4227497728ce33490843d056'},
 {'uuid': '049a3f3a2b5f85cb2971ba77ad66e10c',
  'title': 'Data Coordinator',
  'normalized_job_title': 'data coordinator',
  'parent_uuid': '0b9dd32a367f4562ec77b993053d1910'},
 {'uuid': 'f4b2fb1aa40f661488e2782b6d57ad2f',
  'title': 'Database Developer',
  'normalized_job_title': 'database developer',
  'parent_uuid': 'b90ca4df5690002377a7b0f1f3d40781'},
 {'uuid': '27af8700f5577cec835acee2cb90a2ff',
  'title': 'Data Entry Specialist',
  'normalized_job_title': 'data entry specialist',
  'parent_uuid': 'bf88358c56bb6cbe7eabec38ae333d19'},
 {'uuid': 'c1b670eba9ccb65e7c99f7da116d5b9c',
  'title': 'Database Architect',
  'normalized_job_title': 'database architect',
  'parent_uuid': '596226d5f9ef63de41a852826d483

In [28]:
jobs_json = json.dumps(jobs_id_list)

data_jobs=pd.read_json(my_json_string)

'[{"error": {"code": 404, "message": "Cannot find job with id None"}}, {"uuid": "861a9b9151e11362eb3c77ca914172d0", "title": "Automatic Data Processing Planner", "normalized_job_title": "automatic data processing planner", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "049a3f3a2b5f85cb2971ba77ad66e10c", "title": "Data Coordinator", "normalized_job_title": "data coordinator", "parent_uuid": "0b9dd32a367f4562ec77b993053d1910"}, {"uuid": "f4b2fb1aa40f661488e2782b6d57ad2f", "title": "Database Developer", "normalized_job_title": "database developer", "parent_uuid": "b90ca4df5690002377a7b0f1f3d40781"}, {"uuid": "27af8700f5577cec835acee2cb90a2ff", "title": "Data Entry Specialist", "normalized_job_title": "data entry specialist", "parent_uuid": "bf88358c56bb6cbe7eabec38ae333d19"}, {"uuid": "c1b670eba9ccb65e7c99f7da116d5b9c", "title": "Database Architect", "normalized_job_title": "database architect", "parent_uuid": "596226d5f9ef63de41a852826d483d19"}, {"uuid": "d71c4164c99a5f9bc

In [15]:
data_jobs['normalized_job_code']=data_jobs['uuid']

clean_data_jobs=data_jobs.drop(columns=['error', 'uuid', 'parent_uuid'])

clean_data_jobs

,title,normalized_job_title,normalized_job_code
0,NaN,NaN,NaN
1,Automatic Data Processing Planner,automatic data processing planner,861a9b9151e11362eb3c77ca914172d0
2,Data Coordinator,data coordinator,049a3f3a2b5f85cb2971ba77ad66e10c
3,Database Developer,database developer,f4b2fb1aa40f661488e2782b6d57ad2f
4,Data Entry Specialist,data entry specialist,27af8700f5577cec835acee2cb90a2ff
...,...,...,...
152,Analytical Data Miner,analytical data miner,b0fa6ede410f50b82ab74f5a705fe699
153,Data Analysis Assistant,data analysis assistant,559a21f836c93876f31b60e6d10656a7
154,Data Examination Clerk,data examination clerk,c1fb1a01b78373ac2153c66fa08d16dc
155,Data Entry Clerk,data entry clerk,05bb9a333a66d6eb151e253623efe1c0


In [43]:
result = pd.merge(career, clean_data_jobs, how='left', on=['normalized_job_code'])


In [46]:
def job_code_filtered(data_table):
    codelist = data_table['normalized_job_code'].unique().tolist()
    return codelist

c=job_code_filtered(career)


def api_job_list (c):
    jobs_id_list = []
    for job_code in c:
        req= requests.get(f'http://api.dataatwork.org/v1/jobs/{job_code}')
        jobs_id_list.append(req.json())
    jobs_json= json.dumps(jobs_id_list)
    data_jobs = pd.read_json(jobs_json)
    data_jobs['normalized_job_code'] = data_jobs['uuid']
    dataframe_jobs = data_jobs.drop(columns=['error', 'uuid', 'parent_uuid'])
    return dataframe_jobs

api_job_list(c)



,title,normalized_job_title,normalized_job_code
0,NaN,NaN,NaN
1,Automatic Data Processing Planner,automatic data processing planner,861a9b9151e11362eb3c77ca914172d0
2,Data Coordinator,data coordinator,049a3f3a2b5f85cb2971ba77ad66e10c
3,Database Developer,database developer,f4b2fb1aa40f661488e2782b6d57ad2f
4,Data Entry Specialist,data entry specialist,27af8700f5577cec835acee2cb90a2ff
...,...,...,...
152,Analytical Data Miner,analytical data miner,b0fa6ede410f50b82ab74f5a705fe699
153,Data Analysis Assistant,data analysis assistant,559a21f836c93876f31b60e6d10656a7
154,Data Examination Clerk,data examination clerk,c1fb1a01b78373ac2153c66fa08d16dc
155,Data Entry Clerk,data entry clerk,05bb9a333a66d6eb151e253623efe1c0


In [97]:
country = pd.read_sql_table(table_name='country_info', con=db_connection)
country

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city
...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,Non-Rural
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,city


In [18]:
country.groupby('country_code').count()

,uuid,rural
country_code,,
AT,133,133
BE,179,179
BG,141,141
CY,13,13
CZ,189,189
DE,1420,1420
DK,119,119
EE,18,18
ES,1005,1005


In [114]:
html = requests.get('https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes').content

In [132]:
def country_parsed():
    html = requests.get('https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes').content
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('div', {'class':'col-lg-12 col-md-12 col-sm-12 col-xs-12 content-col content article-content'})[0]
    rows = table.find_all('td')
    rows_parsed = [row.text for row in rows]
    return rows_parsed

c=country_parsed()

def country_df():
    Country_list=[]
    for i in c:
        Country_list.append(re.sub('^\s|[()]|\n','',i))
    Country_clean= [x for x in Country_list if x]
    N = 2
    Country_def = [Country_clean[n:n+N] for n in range(0, len(Country_clean), N)]
    Country_table = pd.DataFrame.from_records(Country_def)
    Country_table_def =Country_table.rename(columns={0:'country_l', 1:'country_code'})
    Country_table_def['country_code']=Country_table_def['country_code'].str.replace('EL', 'GR').str.replace('UK', 'GB')
    return Country_table_def

data=country_df()


pd.set_option('display.max_rows', data.shape[0]+1)


data




,country_l,country_code
0,Belgium,BE
1,Greece,GR
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
5,Spain,ES
6,Luxembourg,LU
7,Romania,RO
8,Czechia,CZ
9,France,FR


In [96]:
result = pd.merge(country, e, how='left', on=['country_code'])
result

,uuid,country_code,rural,country_code2,country_l
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside,AT,Austria
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,AT,Austria
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city,AT,Austria
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country,AT,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city,AT,Austria
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban,SK,Slovakia
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban,SK,Slovakia
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,Non-Rural,SK,Slovakia
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,city,SK,Slovakia


In [ ]:
Country_list=[]
    for i in rows_parsed:
        Country_list.append(re.sub('[()]|\n','',i))
    Country_clean= [x for x in Country_list if x]
    N = 2
    Country_def = [Country_clean[n:n+N] for n in range(0, len(Country_clean), N)]
    return Country_def

In [19]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content


In [20]:
soup = BeautifulSoup(html, 'lxml')



table = soup.find_all('div', {'class':'col-lg-12 col-md-12 col-sm-12 col-xs-12 content-col content article-content'})[0]

table

rows = table.find_all('div', {'id':'bodyContent'})
rows_parsed = [row.text for row in rows]

rows_parsed



['\n\n\n\n\t\t\t\t\tJump to:\t\t\t\t\t\n\t\t\t\t\t\tnavigation\t\t\t\t\t, \n\t\t\t\t\t\tsearch\t\t\t\t\t\n\n\nMember States of the European Union (EU) and other countries have been assigned a two-letter country code, always written in capital letters, and often used as an abbreviation in statistical analyses, tables, figures or maps. \n\nThe protocol order in which countries are often listed is based on the alphabetical list of countries in their national language for EU and EFTA Member States and for candidate countries; for potential candidates, it is based on the alphabetical order of their country code. \nEU Member States come first, followed by European Free Trade Association (EFTA) Member States, candidate countries for EU membership, potential candidates and, finally, other countries. The order in the tables below is first column down, then second column down, etc.. \nEuropean Union (EU)\n\n\n\nBelgium\n\n(BE)\n\nGreece\n\n(EL)\n\nLithuania\n\n(LT)\n\nPortugal\n\n(PT)\n\n\nBulga

In [21]:
rows = table.find_all('td')
rows_parsed = [row.text for row in rows]

rows_parsed





['Belgium\n',
 '(BE)\n',
 'Greece\n',
 '(EL)\n',
 'Lithuania\n',
 '(LT)\n',
 'Portugal\n',
 '(PT)\n',
 'Bulgaria\n',
 '(BG)\n',
 'Spain\n',
 '(ES)\n',
 'Luxembourg\n',
 '(LU)\n',
 'Romania\n',
 '(RO)\n',
 'Czechia\n',
 '(CZ)\n',
 'France\n',
 '(FR)\n',
 'Hungary\n',
 '(HU)\n',
 'Slovenia\n',
 '(SI)\n',
 'Denmark\n',
 '(DK)\n',
 'Croatia\n',
 '(HR)\n',
 'Malta\n',
 '(MT)\n',
 'Slovakia\n',
 '(SK)\n',
 'Germany\n',
 '(DE)\n',
 'Italy\n',
 '(IT)\n',
 'Netherlands\n',
 '(NL)\n',
 'Finland\n',
 '(FI)\n',
 'Estonia\n',
 '(EE)\n',
 'Cyprus\n',
 '(CY)\n',
 'Austria\n',
 '(AT)\n',
 'Sweden\n',
 '(SE)\n',
 'Ireland\n',
 '(IE)\n',
 'Latvia\n',
 '(LV)\n',
 'Poland\n',
 '(PL)\n',
 ' United Kingdom\n',
 ' (UK)\n',
 '\n',
 ' Iceland\n',
 ' (IS)\n',
 '\n',
 ' Norway\n',
 ' (NO)\n',
 ' Liechtenstein\n',
 ' (LI)\n',
 '\n',
 ' Switzerland\n',
 ' (CH)\n',
 ' Montenegro\n',
 ' (ME)\n',
 ' North Macedonia\n',
 ' (MK)\n',
 ' Albania\n',
 ' (AL)\n',
 'Serbia\n',
 ' (RS)\n',
 ' Turkey\n',
 ' (TR)\n',
 ' Bosnia

In [22]:
Country_list=[]

for i in rows_parsed:
    Country_list.append(re.sub('[()]|\n','',i))

Country_list

Country_clean= [x for x in Country_list if x]


N = 2
Country_def = [Country_clean[n:n+N] for n in range(0, len(Country_clean), N)]


Country_def


[['Belgium', 'BE'],
 ['Greece', 'EL'],
 ['Lithuania', 'LT'],
 ['Portugal', 'PT'],
 ['Bulgaria', 'BG'],
 ['Spain', 'ES'],
 ['Luxembourg', 'LU'],
 ['Romania', 'RO'],
 ['Czechia', 'CZ'],
 ['France', 'FR'],
 ['Hungary', 'HU'],
 ['Slovenia', 'SI'],
 ['Denmark', 'DK'],
 ['Croatia', 'HR'],
 ['Malta', 'MT'],
 ['Slovakia', 'SK'],
 ['Germany', 'DE'],
 ['Italy', 'IT'],
 ['Netherlands', 'NL'],
 ['Finland', 'FI'],
 ['Estonia', 'EE'],
 ['Cyprus', 'CY'],
 ['Austria', 'AT'],
 ['Sweden', 'SE'],
 ['Ireland', 'IE'],
 ['Latvia', 'LV'],
 ['Poland', 'PL'],
 [' United Kingdom', ' UK'],
 [' Iceland', ' IS'],
 [' Norway', ' NO'],
 [' Liechtenstein', ' LI'],
 [' Switzerland', ' CH'],
 [' Montenegro', ' ME'],
 [' North Macedonia', ' MK'],
 [' Albania', ' AL'],
 ['Serbia', ' RS'],
 [' Turkey', ' TR'],
 [' Bosnia and Herzegovina', ' BA'],
 [' Kosovo*[1]', ' XK[2]'],
 [' Armenia', ' AM'],
 [' Belarus', ' BY'],
 [' Georgia', ' GE'],
 [' Azerbaijan', ' AZ'],
 [' Moldova', ' MD'],
 [' Ukraine', ' UA'],
 [' Algeria', '

In [23]:
Country_table = pd.DataFrame.from_records(Country_def)

Country_table_n =Country_table.rename(columns={0:'country_l', 1:'country_code'})

Country_table_n

Country_table_n['country_code'].tolist()

['BE',
 'EL',
 'LT',
 'PT',
 'BG',
 'ES',
 'LU',
 'RO',
 'CZ',
 'FR',
 'HU',
 'SI',
 'DK',
 'HR',
 'MT',
 'SK',
 'DE',
 'IT',
 'NL',
 'FI',
 'EE',
 'CY',
 'AT',
 'SE',
 'IE',
 'LV',
 'PL',
 ' UK',
 ' IS',
 ' NO',
 ' LI',
 ' CH',
 ' ME',
 ' MK',
 ' AL',
 ' RS',
 ' TR',
 ' BA',
 ' XK[2]',
 ' AM',
 ' BY',
 ' GE',
 ' AZ',
 ' MD',
 ' UA',
 ' DZ',
 ' LB',
 ' SY',
 ' EG',
 ' LY',
 ' TN',
 ' IL',
 ' MA',
 ' \xa0',
 ' JO',
 ' PS',
 ' \xa0',
 ' RU',
 'AR',
 'CN_X_HK',
 'MX',
 'ZA',
 'AU',
 'HK',
 'NG',
 'KR',
 'BR',
 'IN',
 'NZ',
 'TW',
 'CA',
 'JP',
 'SG',
 'US']

In [24]:
country['country_code2']=country['country_code']

result = pd.merge(country, Country_table_n, how='left', on=['country_code'])

In [25]:
result.groupby('country_code').count()

,uuid,rural,country_code2,country_l
country_code,,,,
AT,133,133,133,133
BE,179,179,179,179
BG,141,141,141,141
CY,13,13,13,13
CZ,189,189,189,189
DE,1420,1420,1420,1420
DK,119,119,119,119
EE,18,18,18,18
ES,1005,1005,1005,1005


In [47]:
result.filter(items=['GR'])


result

,uuid,dem_education_level,dem_full_time_job,normalized_job_code,title,normalized_job_title
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None,NaN,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None,NaN,NaN
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator,data coordinator
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer,database developer
...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,Database Manager,database manager
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None,NaN,NaN
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,Data Officer,data officer
